## Training and predicting objects using Mask RCNN model

1. [Introduction](#Introduction)
2. [Setup](#Setup)
3. [Host pretrained model](#Host-pretrained-model)
4. [Train model](#Train-model)

## Introduction

placeholder

## Setup

Before preparing the data, there are some initial steps required for setup. To train the image classification algorithm on Amazon SageMaker, we need to setup and authenticate the use of AWS services. To begin with, we need an AWS account role with SageMaker access. Here we will use the execution role the current notebook instance was given when it was created.  This role has necessary permissions, including access to your data in S3.

In [2]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

arn:aws:iam::480604769687:role/service-role/AmazonSageMaker-ExecutionRole-20190530T172586


We also need to identify the S3 bucket that you want to use for providing training and validation datasets. It will be used to store the tranied model artifacts as well. In this notebook, we use a default bucket for use with SageMaker in your account. Alternatively, you could use whatever bucket you would like. We use an object prefix to help organize the bucket content.

In [4]:
bucket = sess.default_bucket() # or use your own custom bucket name
prefix = 'DEMO-RCNN-model'

**Important.** Refine and define parameters of your training and inference.

In [15]:
IS_LOCAL = True # defines whether training is done on notebook instance (i.e. IS_LOCAL=TRUE) or on remote Sagemaker nodes (IS_LOCAL=FALSE)

WORK_DIR = "/home/ec2-user/SageMaker/object-detection"

## Prepare dataset

## Install Dependencies

To be able to run pre-trained MRCNN model, we need to resolve several dependencies:
- install number of Python packages;
- build MRCNN package from source;
- copy model weights trained on MS COCO dataset.

Review setup.sh and requirements.txt for details.

In [6]:
! pygmentize code/setup.sh

#!/bin/bash
# Download and install Mask_RCNN implementations
git clone https://github.com/matterport/Mask_RCNN
cd Mask_RCNN
python setup.py install

# Download pretrained model weights for COCO dataset
wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
    
# Install pycocotools to work with MS COCO data set. 
git clone https://github.com/pdollar/coco.git
cd coco/PythonAPI
make
sudo make install
sudo python setup.py install


In [8]:
! pygmentize code/requirements.txt

numpy
scipy
Pillow
cython
matplotlib
scikit-image
tensorflow>=1.3.0
keras>=2.0.8
opencv-python
h5py
imgaug
IPython[all]


In [ ]:
if IS_LOCAL: # if training on notebook instance, ensure that all dependecies are deployed.
    ! sudo sh {WORK_DIR}/code/setup.sh {WORK_DIR}
    ! pip install -r {WORK_DIR}/code/requirements.txt

In [27]:
dir = "code"
! ls -l {dir}

total 16
-rw-rw-r-- 1 ec2-user ec2-user  13 Sep 18 03:36 lab1-inference.py
-rw-rw-r-- 1 ec2-user ec2-user  12 Sep 17 03:22 lab1-training.py
-rw-rw-r-- 1 ec2-user ec2-user 119 Sep 18 13:28 requirements.txt
-rw-rw-r-- 1 ec2-user ec2-user 638 Sep 19 18:28 setup.sh


## Inference of pre-trained model